# 第14回　エキゾチック・オプションのプライシング例 

## はじめに

```
ここまでの例はほとんどすべて、理論的に正解が別にちゃんと分かるものとか、他の代替的な方法でもほとんど同じ結果になるとかいった例ばかりでしたから。
そこで、この講座の最後に、「これこそモンテカルロ・シミュレーションが必要だ！」という例をご紹介して終わりたい
```

## プライシング対象オプション

```
（１）　アベレージ・オプション
　コール／プット　：　コール
　満期　：　１年
　行使価格　：　毎月末のドル円為替レートの平均値

（２）　ルックバック・オプション
　コール／プット　：　コール
　満期　：　１年
　行使価格　：　毎月末のドル円為替レートの最小値
※いずれも、行使価格の決定には現在の為替レートは含まないものとします。

＜前提条件＞
　現在のドル円為替レート　：　１ドル ＝ 80 円
　円金利　：　0.2％
　ドル金利　：　0.5％
　ボラティリティ　：　10％
```

## 原資産価格のパスの作成

### プライシングの基本的考え方

- 原理はプレーンな通貨オプションの場合と全く同じで、「リスク中立経済下におけるオプションの将来価値の期待値」を現在価値に引き直せばよい
- 問題は、オプションの行使価格が為替レートの満期までの価格推移に基づいて決まるため、到底、オプション価値の期待値が解析的には求まりそうもないこと
- モンテカルロ・シミュレーションの出番
- シミュレーションによりオプション価値の平均値を求め、それを現在価値にすれば、リスク中立評価法の原理からほぼ正しい値になっているはず
- オプションの条件から、満期まで１ヶ月ごとの為替レート情報が必要です。 したがって、プレーンなオプションの場合と違い、１ヶ月ごとの為替レートを出力していくようなシミュレーションが必要になります
- 後は、基本となるモデル式の形やリスク中立確率下における μ の設定など、すべてプレーンな通貨オプションの場合と同じ


## BS式とシミュレーション

- $t$
  - その時点までの年数。
  - ${\frac{1}{12}}$ごと
- $Z$
  - 1ヶ月ごとのブラウン運動による増減
    - 12章では1年。14章では1ヶ月ごと → 分散を $\frac{1}{12}$
- $\Delta W$
  - $\sqrt{\frac{1}{12}}Z$
- $Wt$
  - $\Delta W$の合算値
  - その時点までのブラウン運動の履歴
- $S_T = S_{0}e^{\mu T+{\sigma}W_{T}}$
  - 1ヶ月ごとの為替レート
- ${\mu} = r - q - \frac{\sigma^2}{2}$
  - リスク中立経済下における期待値

### シミュレーション

${\mu} = r - q - \frac{\sigma^2}{2}$

$S_T = S_{0}e^{uT+{\sigma}W_{T}}$

$S_1 = S_{0}e^{u1+{\sigma}W_{T}}$

In [29]:
import numpy as np

d_rate = 0.005
print("ドル金利 は " + str(d_rate))

y_rate = 0.002
print("円金利 は " + str(y_rate))

m = 0.002 - 0.005 - (0.1 ** 2) / 2
print("mu は " + str(m))

S0 = 80
print("S0 は " + str(S0))

v = 0.1
print("ボラティリティ は " + str(v))

#sim_count = 12
#print("シミュレーション回数 は " + str(sim_count))

# tを設定(1/12から1までの12個の等差数列)
t = np.linspace(1/12, 1, 12)

# 標準正規分布に従う乱数をsim_count個生成
z = np.random.normal(0, 1, 12)
# 乱数の先頭5個を表示
print("乱数のサンプル" + str(z[:5]))

# deita_tを設定(z * sqrt(1/12))
delta_t = z * np.sqrt(1/12)
print("delta_tのサンプル" + str(delta_t[:5]))

# Wtを設定(0からcumsum(delta_t)までの累積和)
Wt = np.cumsum(delta_t)
print("Wtのサンプル" + str(Wt[:5]))

# Stを設定(S0 * eの(m * t + v * Wt)乗)
St = S0 * np.exp(m * t + v * Wt)
print("Stのサンプル" + str(St[:5]))
# アベレージオプションの行使価格
average_strike = np.mean(St)
print("アベレージオプションの行使価格は " + str(average_strike))
# ルックバックオプションの行使価格
lookback_strike = np.min(St)
print("Stの最小値は " + str(lookback_strike))
# 行使日の為替レート(t=12)
exercise_rate = St[-1]
print("行使日の為替レートは " + str(exercise_rate))

# 満期時のオプション価値
# アベレージオプションの価値は、行使日の価格とアベレージオプションの行使価格を比較して、その差が正ならばその差を、負ならば0を選ぶ
option_value = np.maximum(exercise_rate - average_strike, 0)
print("アベレージオプション価値は " + str(option_value))
# ルックバックオプションの価値は、行使日の価格とルックバックオプションの行使価格を比較して、その差が正ならばその差を、負ならば0を選ぶ
option_value = np.maximum(exercise_rate - lookback_strike, 0)
print("ルックバックオプション価値は " + str(option_value))



ドル金利 は 0.005
円金利 は 0.002
mu は -0.008
S0 は 80
ボラティリティ は 0.1
乱数のサンプル[ 1.44450005 -0.86090325 -0.71241264 -0.90502609 -0.06557615]
delta_tのサンプル[ 0.41699125 -0.24852136 -0.20565582 -0.26125853 -0.0189302 ]
Wtのサンプル[ 0.41699125  0.16846989 -0.03718593 -0.29844446 -0.31737466]
Stのサンプル[83.3508738  81.25076933 79.54381816 77.44093543 77.24296434]
アベレージオプションの行使価格は 77.75622470618201
Stの最小値は 73.83998381411834
行使日の為替レートは 78.08535358006876
アベレージオプション価値は 0.32912887388674505
ルックバックオプション価値は 4.245369765950414


In [95]:
# ここまでの計算を1000借り換えして、オプション価値の平均を求める
# シミュレーション回数
sim_count = 1000000
# 1回の計算で実行するシミュレーション回数
batch_size = 100000
# 総処理数
total_processed = 0
d_rate = 0.005
y_rate = 0.002
# mu
m = y_rate - d_rate - (0.1 ** 2) / 2
# 現在の為替レート
S0 = 80
# ボラティリティ
v = 0.1
# 1年の月数(分割数。今回は12で固定)
divisions_in_year = 12
# シミュレーション結果を格納する配列
average_option_values = np.zeros(sim_count)
lookback_option_values = np.zeros(sim_count)
# tを設定(1/12から1までの12個の等差数列)。
t = np.linspace(1/divisions_in_year, 1, divisions_in_year)

# batch_sizeごとに繰り返し
while total_processed < sim_count:
    # batch_size分の乱数を生成する。1計算ごとにdivision_in_year回の乱数を生成する
    # 乱数はbatch_size * divisions_in_year個生成する
    z = np.random.normal(0, 1, batch_size * divisions_in_year)
    # 乱数を12個ごとに分割する
    z = z.reshape(batch_size, divisions_in_year)
    delta_t = z * np.sqrt(1/divisions_in_year)
    Wt = np.cumsum(delta_t, axis=1)
    # Stを設定(S0 * eの(m * t + v * Wt)乗)
    St = S0 * np.exp(m * t + v * Wt)
    # アベレージオプションの行使価格
    average_strike = np.mean(St, axis=1)
    # ルックバックオプションの行使価格
    lookback_strike = np.min(St, axis=1)
    # 行使日の為替レート(t=12)
    exercise_rate = St[:, -1]
    # 満期時のオプション価値
    # アベレージオプションの価値を計算。batch_size回の計算結果をbatchサイズのIndex以降に追加する
    average_option_values[total_processed:total_processed+batch_size] = np.maximum(exercise_rate - average_strike, 0)
    # ルックバックオプションの価値を計算
    lookback_option_values[total_processed:total_processed+batch_size] = np.maximum(exercise_rate - lookback_strike, 0)
    total_processed += batch_size
    
# オプション価値の平均を求める
average_option_value = np.mean(average_option_values)
lookback_option_value = np.mean(lookback_option_values)
print("アベレージオプション価値は " + str(average_option_value))
print("ルックバックオプション価値は " + str(lookback_option_value))
# オプションの現在価値。割引率は0.002
discount_rate = 0.002
average_option_value = average_option_value * np.exp(-discount_rate)
lookback_option_value = lookback_option_value * np.exp(-discount_rate)
print("アベレージオプションの現在価値は " + str(average_option_value))
print("ルックバックオプションの現在価値は " + str(lookback_option_value))
# オプション価値の標準偏差
average_option_value_std = np.std(average_option_values) * np.exp(-d_rate) / np.sqrt(sim_count)
lookback_option_value_std = np.std(lookback_option_values) * np.exp(-d_rate) / np.sqrt(sim_count)
print("アベレージオプション価値の標準偏差は " + str(average_option_value_std))
print("ルックバックオプション価値の標準偏差は " + str(lookback_option_value_std))





アベレージオプション価値は 1.6738172910551963
ルックバックオプション価値は 4.696114930601629
アベレージオプションの現在価値は 1.670473001877027
ルックバックオプションの現在価値は 4.68673208671193
アベレージオプション価値の標準偏差は 0.0026333793102718095
ルックバックオプション価値の標準偏差は 0.004753647573693399
